<a href="https://colab.research.google.com/github/bongkyunSON/Machine-Learning/blob/main/ML_project_final_221125.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Module Import
### Module Version

<Module Version>

- numpy              1.23.4
- pandas             1.5.1
- scikit-learn       1.1.3
- imbalanced-learn   0.9.1
- xgboost            1.7.1
- lightgbm           3.3.3
- optuna             2.10.1
- tqdm               4.64.1


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install jtlearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement jtlearn (from versions: none)
ERROR: No matching distribution found for jtlearn


In [ ]:
from sklearn.decomposition import PCA # 차원축소
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, f1_score, precision_score, recall_score

from imblearn.under_sampling import * # 임벨런스
from imblearn.over_sampling import * # 임벨런스
from imblearn.combine import * # 임벨런스

import os
import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from patsy import dmatrices
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

# from jtlearn import Preprocessing
# from ensemble import BinaryCalssifier, Regressor

### Load Data

In [ ]:
# Load Data
save_path = 'submission/'
base_path = 'data/'

train = pd.read_csv(base_path + 'train.csv')
test = pd.read_csv(base_path + 'test.csv')
submission = pd.read_csv(base_path + 'sample_submission.csv')

FileNotFoundError: ignored

### Preprocessing

1. Fill/Drop Null data
2. Standard Scaling
3. Ordinal Encoding

In [ ]:
train = train.fillna(train.mean())

X = train.drop(columns=["ID", "Y_LABEL"])
y = train["Y_LABEL"]
test = test.drop(columns=['ID'])

train_num_cols = X.drop(columns=['COMPONENT_ARBITRARY']).columns.tolist()
test_num_cols = test.drop(columns=['COMPONENT_ARBITRARY']).columns.tolist()

ss = StandardScaler()
ss2 = StandardScaler()

ss2.fit(X[test_num_cols])
X[train_num_cols] = ss.fit_transform(X[train_num_cols])
test[test_num_cols] = ss2.transform(test[test_num_cols])

X.COMPONENT_ARBITRARY = X.COMPONENT_ARBITRARY.map({"COMPONENT1" : 1, "COMPONENT2" : 2, "COMPONENT3" : 3, "COMPONENT4" : 4})
test.COMPONENT_ARBITRARY = test.COMPONENT_ARBITRARY.map({"COMPONENT1" : 1, "COMPONENT2" : 2, "COMPONENT3" : 3, "COMPONENT4" : 4})

X, X_test, y, y_test = train_test_split(X, y, test_size=0.2, random_state=172)
X_test = X_test[test.columns]

In [ ]:
test

In [ ]:
y

## Regression
### Multicollinearity

In [ ]:
empty_cols = X.drop(columns=test.columns).columns
X_reg = X[test.columns]

for col in empty_cols:
    y_reg = X[col]
    X_reg['intercept'] = 1
    lm = sm.OLS(y_reg, X_reg)
    results = lm.fit()

    print(f"***************************** {col} *****************************\n")
    print(results.summary())

X_reg = X_reg.drop(columns=['intercept'])

In [ ]:
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X_reg.values, i) for i in range(len(X_reg.columns))]
vif["features"] = X_reg.columns
vif

In [ ]:
X_reg

### regressor hyper parameters

{

"metric": , # 평가지표(default='r2_score')

"learner": , # 학습모델['rf', 'xgb', 'lgbm'] 중 아무 조합 선택 ㄱㄱ(default=['rf', 'xgb', 'lgbm'])

"ensemble": , # ['voting', 'stacking'](default='voting')

"learning_rate": , # 학습률(default=0.05)

"random_state": , # 난수 seed(default=42)

"early_stopping_rounds": , # overfitting 방지용(default=10)

"optimize": , # optuna 사용할지 말지 True or False 사용 ㄱㄱ(default=False)

"n_trials": , # optuna 횟수(default=20)

"cv": , # K-fold CV의 K(default=5)

"N": , # voting에서 모델별 weights의 조합가지수(default=5)

}

### Find Best Regressor For Each Column

In [ ]:
learners = [['rf']]
# learners = [['rf'], ['xgb'], ['lgbm'],
#             ['rf', 'xgb'], ['rf', 'lgbm'], ['xgb', 'lgbm'], 
#             ['rf', 'xgb', 'lgbm']]

reg_dict = {}

df_score_train = pd.DataFrame([[0]*len(empty_cols)]*len(learners))
df_score_train.columns = empty_cols
df_score_train.index = ['+'.join(learner) for learner in learners]

df_score_val = copy.deepcopy(df_score_train)
print("empty column length is", len(empty_cols))

for col in empty_cols:
    reg_list = []
    score_list = []
    temp_score_val = []
    print(f"************************* {col} *************************\n")
    for learner in learners:
        print(f"--------------- {'+'.join(learner)} ---------------\n")
        reg = Regressor(learner=learner,
                        ensemble='stacking',
                        metric='mse',
                        optimize=False,
                        n_trials=10,
                        cv=5,
                        N=5)
        y_reg = X[col]
        # train-validation split
        X_reg_train, X_reg_val, y_reg_train, y_reg_val = train_test_split(X_reg, y_reg, test_size=0.3, random_state=69)

        # model fitting
        reg.fit(X_reg_train, y_reg_train) # n_trials: optuna 조지는 정도 / cv: K-fold의 K값

        # prediction
        y_reg_train_pred = reg.predict(X_reg_train)
        y_reg_val_pred = reg.predict(X_reg_val)

        # scoring
        score_train = reg.score(y_reg_train, y_reg_train_pred)
        score_val = reg.score(y_reg_val, y_reg_val_pred)
        
        reg_list.append(reg)
        score_list.append(score_val)
        df_score_train.loc['+'.join(learner), col] = score_train
        df_score_val.loc['+'.join(learner), col] = score_val
    
    max_idx = score_list.index(max(score_list))
    reg_dict[col] = reg_list[max_idx]

In [ ]:
df_score_train

In [ ]:
df_score_val

In [ ]:
drop_reg_cols = []
final_reg_cols = []
for col in df_score_val.columns:
    if df_score_val[col].max() < 0.5:
        drop_reg_cols.append(col)
    elif df_score_val[col].max() > 0.7:
        final_reg_cols.append(col)

print(drop_reg_cols)
print(final_reg_cols)

회귀성능 잘 나오는 columns

['FNOX', 'FOXID', 'FSO4', 'FTBN', 'S', 'SB', 'SI', 'V100']

In [ ]:
X2 = X.drop(columns=drop_reg_cols)

##
X3 = X2[test.columns]
temp_test = copy.deepcopy(test)
X_temp_test = copy.deepcopy(X_test)

for col, reg in reg_dict.items():
    if col not in drop_reg_cols:
        print(col)
        X2[col] = reg.predict(X3)
        test[col] = reg.predict(temp_test)
        X_test[col] = reg.predict(X_temp_test)

In [ ]:
X2

### Imbalanced Sampling

{

"under": {


    'RandomUnderSampler': RandomUnderSampler,

    'TomekLinks': TomekLinks,

    'CondensedNearestNeighbour': CondensedNearestNeighbour, 

    'OneSidedSelection': OneSidedSelection,

    'EditedNearestNeighbours': EditedNearestNeighbours,

    'NeighbourhoodCleaningRule': NeighbourhoodCleaningRule

},

"over": {

    'RandomOverSampler': RandomOverSampler,

    'SMOTE': SMOTE,

    'ADASYN': ADASYN,

    'NeighbourhoodCleaningRule': NeighbourhoodCleaningRule

},

"hybrid": {

    'SMOTEENN': SMOTEENN,

    'SMOTETomek': SMOTETomek

}

}

In [ ]:
def sampling_group(X, y,
                   categorical_feature: str,
                   use_sampler: bool=True) -> dict: 
    """
    divide train_df to make each group df
    return grouped df list 
    """

    # concat X2 and y2 to divide groups
    if hasattr(y, 'name'):
        y_col = y.name
    else:
        y_col = 'Y_LABEL'

    concat_df: pd.DataFrame = pd.concat([X,y], axis=1)
    group_dic = {}

    for criteria in sorted(concat_df[categorical_feature].unique()): 
        print(f"dividing my df on {criteria}")
        temp_df = concat_df.loc[concat_df[categorical_feature] == criteria,].drop(columns=categorical_feature)

        # make grouped X, y
        X2 = temp_df.drop(columns=[y_col])
        y2 = temp_df[y_col]

        group_dic.update({criteria: (X2, y2)})

    return group_dic

In [ ]:
# sampler parameter
variable_dict = {
    "test_size": 0.1, 
    "learner": 'rf',
    "objective": "under",
    "sampler": "RandomUnderSampler",
    "random_state_": 42,
    "dimensionality": PCA
}

balancing = Preprocessing(**variable_dict)

# 샘플링 그룹핑 스플릿
grouped_dict = balancing.sampling_group(X2, y, categorical_feature='COMPONENT_ARBITRARY')
split_X_y_bundle = balancing.split_X_y_bundle(grouped_dict)
print()

In [ ]:
grouped_dict[1][0]

In [ ]:
test

### Dimensionality(Feature Importance)

In [ ]:
import seaborn as sns
def choose_drop_features(feature_importance_dict, threshold: int=0, draw: bool=False):

    if draw == True:
        for criteria, feature_importance in feature_importance_dict.items(): 
            plt.figure(figsize=(12, 6))
            plt.title(f'{criteria} Feature Importances')
            sns.barplot(x=feature_importance, y=feature_importance.index)
            plt.show()
                
    drop_target_dict = {}
    for criteria, feature_importance in feature_importance_dict.items():
        temp_df = feature_importance.reset_index()
        temp_df.columns = ["name", "value"]

        drop_target_dict[criteria] = temp_df[temp_df.value <= threshold].name.to_list()

    return drop_target_dict

In [ ]:
features_dict = balancing.feature_importance_for_groups(split_X_y_bundle)
drop_target_dict = choose_drop_features(features_dict, draw=True)

print()
print(drop_target_dict)
print()

['FNOX', 'FOXID', 'FSO4', 'FTBN', 'S', 'SB', 'SI', 'V100']

### Classification

 - classifier hyper parameters
 
{

"metric": , # 평가지표(default='r2_score')

"learner": , # 학습모델['rf', 'xgb', 'lgbm'] 중 아무 조합 선택 ㄱㄱ(default=['rf', 'xgb', 'lgbm'])

"ensemble": , # ['voting', 'stacking'](default='voting')

"learning_rate": , # 학습률(default=0.05)

"random_state": , # 난수 seed(default=42)

"early_stopping_rounds": , # overfitting 방지용(default=10)

"optimize": , # optuna 사용할지 말지 True or False 사용 ㄱㄱ(default=False)

"n_trials": , # optuna 횟수(default=20)

"cv": , # K-fold CV의 K(default=5)

"N": , # voting에서 모델별 weights의 조합가지수(default=5)

}

In [ ]:
test_final = pd.DataFrame()
val_final = pd.DataFrame()
X_y_union = pd.concat([X_test, y_test], axis=1)

for criterion, drop_target_list in drop_target_dict.items():
    X_train, X_val, y_train, y_val = split_X_y_bundle[criterion]
    
    X_train = X_train.drop(columns=drop_target_list)
    X_val = X_val.drop(columns=drop_target_list)
    
    test_temp = test[test.COMPONENT_ARBITRARY == criterion].drop(columns=drop_target_list)
    test_temp = test_temp.drop(columns=['COMPONENT_ARBITRARY'])
    
    X_test_temp = X_y_union[X_y_union.COMPONENT_ARBITRARY == criterion].drop(columns=drop_target_list)
    X_test_temp = X_test_temp.drop(columns=['COMPONENT_ARBITRARY', 'Y_LABEL'])
    
    y_test_temp = X_y_union[X_y_union.COMPONENT_ARBITRARY == criterion].Y_LABEL
    
    # Syncronize columns order between X3 and test
    test_temp = test_temp[X_train.columns]
    X_test_temp = X_test_temp[X_train.columns]
    
    # model initializing
    clf = BinaryCalssifier(metric='f1_score',
                           learner=['rf'],
                           optimize=False,
                           n_trials=20,
                           cv=5,
                           N=5,
                           random_state=12)
    # model training
    clf.fit(X_train, y_train) # n_trials: optuna 조지는 정도 / cv: K-fold의 K값

    # prediction
    y_train_pred = clf.predict(X_train)
    y_val_pred = clf.predict(X_val)
    y_test_pred = clf.predict(X_test_temp)
    
    # scoring
    score_train = clf.score(y_train, y_train_pred)
    score_val = clf.score(y_val, y_val_pred)
    score_test = clf.score(y_test_temp, y_test_pred)
    
    print(f'******************** Component {criterion} ********************')
    print("Train F1_score is %.4f" % (score_train))
    print("Validation F1_score is %.4f" % (score_val))
    print("Test F1_score is %.4f" % (score_test))

    # fill prediction value in test data
    test_temp['Y_LABEL'] = clf.predict(test_temp)
    test_final = pd.concat([test_final, test_temp], axis=0)

    # fill prediction value in dummy test data
    X_test_temp['Y_LABEL'] = y_test_pred
    val_final = pd.concat([val_final, X_test_temp], axis=0)
    conf_matrix_val = confusion_matrix(y_val, y_val_pred)
    conf_matrix_test = confusion_matrix(y_test_temp, y_test_pred)
    
    print('-' * 53)
    print("")
    print(conf_matrix_val)
    print('clf precision:\t%.4f' % (precision_score(y_val, y_val_pred)))
    print('clf recall:\t%.4f' % (recall_score(y_val, y_val_pred)))
    print('clf f1_score:\t%.4f' % (f1_score(y_val, y_val_pred)))
    print('-' * 53)
    print("")
    cm_display = ConfusionMatrixDisplay(confusion_matrix = conf_matrix_test,
                                        display_labels = ['Normal', 'Anormal'])
    cm_display.plot()
    plt.show()
    print('Final model precision:\t%.4f' % (precision_score(y_test_temp, y_test_pred)))
    print('Final model recall:\t%.4f' % (recall_score(y_test_temp, y_test_pred)))
    print('Final model f1_score:\t%.4f' % (f1_score(y_test_temp, y_test_pred)))
    print('*' * 53)

test_final = test_final.sort_index()
val_final = val_final.sort_index()
y_pred = val_final.Y_LABEL.values

In [ ]:
val_final

In [ ]:
test_final

### Confusion Matrix

In [ ]:
# conf_matrix = confusion_matrix(y_val, y_pred)

### Classification Metrics

In [ ]:
# print('*' * 40)
# print('Final model precision:\t%.4f' % (precision_score(y_test, y_pred)))
# print('Final model recall:\t%.4f' % (recall_score(y_test, y_pred)))
# print('Final model f1_score:\t%.4f' % (f1_score(y_test, y_pred)))
# print('*' * 40)

### Make submission file

In [ ]:
if 'submission_oil.csv' in os.listdir(save_path):
    count = 0
    for name in os.listdir(save_path):
        if 'submission_oil' in name:
            count += 1
    filename = f"submission_oil{count + 1}.csv"
else:
    filename = 'submission_oil.csv'

# Export submission file
submission.Y_LABEL = test_final.Y_LABEL
submission.to_csv(save_path + filename, index=False)